In [1]:
import json
import os, sys
import _jsonnet
from tqdm.notebook import tqdm

from nltk.translate.bleu_score import corpus_bleu
import numpy as np
import random

In [2]:
sys.path.append(os.path.abspath('/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/third_party/wikisql'))
sys.path.append(os.path.abspath('/Users/mac/Desktop/syt/Deep-Learning/Projects-M'))

In [3]:
from ratsql.commands.infer import Inferer
from ratsql.datasets.spider import SpiderItem
from ratsql.utils import registry

In [4]:
from SpeakQL.SpeakQL.Allennlp_models.utils.spider import process_sql, evaluation

In [5]:
import torch

In [ ]:
exp_config_path = '/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/experiments/spider-glove-run.jsonnet'
exp_config = json.loads(_jsonnet.evaluate_file(exp_config_path))
exp_config

In [ ]:
root_dir = '/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql'
model_config_path = os.path.join(root_dir, exp_config["model_config"])
model_config_args = exp_config.get("model_config_args")
model_config_args

In [ ]:
infer_config = json.loads(_jsonnet.evaluate_file(model_config_path, tla_codes={'args': json.dumps(model_config_args)}))
infer_config

In [ ]:
checkpoint_step = 40000
model_dir = '/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/logdir/glove_run/bs=20,lr=7.4e-04,end_lr=0e0,att=0'

inferer = Inferer(infer_config)
inferer.device = torch.device("cpu")
model = inferer.load_model(model_dir, checkpoint_step)
dataset = registry.construct('dataset', inferer.config['data']['val'])

for _, schema in dataset.schemas.items():
    model.preproc.enc_preproc._preprocess_schema(schema)

In [6]:
def Load_Rat_sql(root_dir,
                 exp_config_path,
                 model_dir,
                 checkpoint_step=40000):

    exp_config = json.loads(_jsonnet.evaluate_file(exp_config_path))
    
    model_config_path = os.path.join(root_dir, exp_config["model_config"])
    model_config_args = exp_config.get("model_config_args")
    
    infer_config = json.loads(_jsonnet.evaluate_file(model_config_path, tla_codes={'args': json.dumps(model_config_args)}))

    inferer = Inferer(infer_config)
    inferer.device = torch.device("cpu")
    model = inferer.load_model(model_dir, checkpoint_step)
    dataset = registry.construct('dataset', inferer.config['data']['val'])

    for _, schema in dataset.schemas.items():
        model.preproc.enc_preproc._preprocess_schema(schema)
    
    _ret_dict = {
        'model': model,
        'dataset': dataset,
        'inferer': inferer,
    }
    
    return _ret_dict
    

In [7]:
rat_sql_model_dict = Load_Rat_sql(root_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql',
                                  exp_config_path='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/experiments/spider-glove-run.jsonnet',
                                  model_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/logdir/glove_run/bs=20,lr=7.4e-04,end_lr=0e0,att=0',
                                  checkpoint_step=40000)
rat_sql_model_dict.keys()


WARNING <class 'ratsql.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}
WARNING <class 'ratsql.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'clause_order': None, 'end_with_from': True, 'factorize_sketch': 2, 'include_literals': False, 'infer_from_conditions': True, 'name': 'spider', 'output_from': True, 'use_table_pointer': True}, 'max_count': 5000, 'min_freq': 4, 'save_path': 'data/spider/nl2code-glove,cv_link=true', 'use_seq_elem_rules': True}, 'encoder_preproc': {'compute_cv_link': True, 'compute_sc_link': True, 'count_tokens_in_word_emb_for_vocab': True, 'db_path': 'data/spider/database', 'fix_issue_16_primary_keys': True, 'include_table_name_in_column': False, 'max_count': 5000, 'min_freq': 4, 'save_path': 'data/spider/nl2code-glove,cv_link=true', 'word_emb': {'kind': '42B', 'lemmatize': True, 'name': 'glove'}}}


Loading model from /Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/logdir/glove_run/bs=20,lr=7.4e-04,end_lr=0e0,att=0/model_checkpoint-00040000


DB connections: 100%|██████████| 166/166 [00:01<00:00, 132.91it/s]


dict_keys(['model', 'dataset', 'inferer'])

In [ ]:
rat_sql_asr_model_dict = Load_Rat_sql(root_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql',
                                      exp_config_path='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/experiments/spider-glove-ASR-run.jsonnet',
                                      model_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/logdir/glove_ASR_run/ASR,bs=20,lr=7.4e-04,end_lr=0e0,att=0',
                                      checkpoint_step=40000)
rat_sql_asr_model_dict.keys()


In [ ]:
def Question(q, db_id, model_dict=rat_sql_model_dict):
    model = model_dict['model']
    dataset = model_dict['dataset']
    inferer = model_dict['inferer']
    
    spider_schema = dataset.schemas[db_id]
    data_item = SpiderItem(
        text=None,  # intentionally None -- should be ignored when the tokenizer is set correctly
        code=None,
        schema=spider_schema,
        orig_schema=spider_schema.orig,
        orig={"question": q}
    )
    model.preproc.clear_items()
    enc_input = model.preproc.enc_preproc.preprocess_item(data_item, None)
    preproc_data = enc_input, None
    with torch.no_grad():
        return inferer._infer_one(model, data_item, preproc_data, beam_size=1, use_heuristic=True)

In [ ]:
Question("how many singers do we have?", "concert_singer")

In [ ]:
Question("display the employee i D and salary of all employees who report to pye um, first name.", "hr_1")

In [ ]:
Question("how many singers do we have?", "concert_singer", model_dict=rat_sql_asr_model_dict)

In [ ]:
Question("display the employee i D and salary of all employees who report to pye um, first name.", "hr_1",
         model_dict=rat_sql_asr_model_dict)

### Evaluate the trained model

In [ ]:
dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'

In [ ]:
with open(dev_path, 'r') as f:
    dev_dataset = json.load(f)
len(dev_dataset)

In [ ]:
d = dev_dataset[0]
d

In [ ]:
question(d['question'], d['db_id'])

In [ ]:
pred_sql_list = []

for d in tqdm(dev_dataset):
    pred = question(d['question'], d['db_id'])[0]
    pred_sql_list.append(pred['inferred_code'])

len(pred_sql_list)

In [ ]:
dev_pred_path = './output/dev_output.txt'

In [ ]:
# with open(dev_pred_path, 'w') as f:
#     for pred in pred_sql_list:
#         f.write(pred + '\n')

### Predict SQL given transcribed dataset JSON files

In [ ]:
DATASET = 'train'
input_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/{0}/{0}_asr_amazon.json'.format(DATASET)
output_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/{0}/{0}_asr_amazon_RatsqlPredicted.json'.format(DATASET)


In [ ]:
# Paths for human test 
input_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_asr_amazon.json'
output_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_asr_amazon_RatsqlPredicted.json'


In [ ]:
with open(input_dataset_path, 'r') as f:
    asr_dataset = json.load(f)
len(asr_dataset)

In [ ]:
asr_dataset[0].keys()

In [ ]:
for i, d in tqdm(enumerate(asr_dataset), total=len(asr_dataset)):
    if 'ratsql_pred_sql' in d:
        continue
        
    pred = Question(d['question'], d['db_id'])
    if len(pred) == 0:
        print('{}: question({}, {}) failed'.format(i, d['question'], d['db_id']))
        d['ratsql_pred_sql'] = ''
    else:
        d['ratsql_pred_sql'] = pred[0]['inferred_code']

asr_dataset[0]

In [ ]:
d

In [ ]:
pred = question(d['question'], d['db_id'])
pred

In [ ]:
# with open(output_dataset_path, 'w') as f:
#     json.dump(asr_dataset, f, indent=4)

### Predict & Evaluate SQL given rewriter output / original / ASR cands

In [ ]:
tables_json = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/tables.json'
kmaps = evaluation.build_foreign_key_map_from_json(tables_json)

In [ ]:
def EvaluateSQL(pred_str,
                gold_str,
                db,
                db_dir='/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/database/'):
    
    db_path = os.path.join(db_dir, db, db + ".sqlite")
    schema = process_sql.Schema(process_sql.get_schema(db_path))
    try:
        g_sql = process_sql.get_sql(schema, gold_str)  # Train #3153/18259, in 'assets_maintenance', 'ref_company_types' not found 
        p_sql = process_sql.get_sql(schema, pred_str)
    except:
        print('{}\n{}\n{}\nprocess_sql.get_sql() failed'.format(pred_str, gold_str, db))
        return 0, 0
    
    # Rebuilding... copied from official evaluate 
    kmap = kmaps[db]
    g_valid_col_units = evaluation.build_valid_col_units(g_sql['from']['table_units'], schema)
    g_sql = evaluation.rebuild_sql_val(g_sql)
    g_sql = evaluation.rebuild_sql_col(g_valid_col_units, g_sql, kmap)
    p_valid_col_units = evaluation.build_valid_col_units(p_sql['from']['table_units'], schema)
    p_sql = evaluation.rebuild_sql_val(p_sql)
    p_sql = evaluation.rebuild_sql_col(p_valid_col_units, p_sql, kmap)
    
    evaluator = evaluation.Evaluator()
    exact_match = evaluator.eval_exact_match(p_sql, g_sql)   # will modify p_sql, g_sql
    partials = evaluator.partial_scores
    partial_summary_score = sum([partials[tp]['f1'] * max(partials[tp]['label_total'], partials[tp]['pred_total']) for tp in partials]) / sum([max(partials[tp]['label_total'], partials[tp]['pred_total']) for tp in partials])

    return int(exact_match), partial_summary_score

In [ ]:
def EvaluateSQL_full(glist,
                     plist,
                     db_id_list,
                     kmaps,
                     db_dir='/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/database/'):
    # Only using 'match', not using 'exec'
    etype = 'match'
    
    # with open(gold) as f:
    #     glist = [l.strip().split('\t') for l in f.readlines() if len(l.strip()) > 0]

    # with open(predict) as f:
    #     plist = [l.strip().split('\t') for l in f.readlines() if len(l.strip()) > 0]
    
    # plist = [("select max(Share),min(Share) from performance where Type != 'terminal'", "orchestra")]
    # glist = [("SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != 'Live final'", "orchestra")]
    evaluator = evaluation.Evaluator()

    levels = ['easy', 'medium', 'hard', 'extra', 'all']
    partial_types = ['select', 'select(no AGG)', 'where', 'where(no OP)', 'group(no Having)',
                     'group', 'order', 'and/or', 'IUEN', 'keywords']
    entries = []
    scores = {}

    for level in levels:
        scores[level] = {'count': 0, 'partial': {}, 'exact': 0., 'partial_summary': 0.}
        scores[level]['exec'] = 0
        for type_ in partial_types:
            scores[level]['partial'][type_] = {'acc': 0., 'rec': 0., 'f1': 0.,'acc_count':0,'rec_count':0}

    eval_err_num = 0
    for p, g, db in tqdm(zip(plist, glist, db_id_list), total=len(plist)):
        p_str = p
        g_str = g
        db_name = db
        db = os.path.join(db_dir, db, db + ".sqlite")
        schema = process_sql.Schema(process_sql.get_schema(db))
        g_sql = process_sql.get_sql(schema, g_str)
        hardness = evaluator.eval_hardness(g_sql)
        scores[hardness]['count'] += 1
        scores['all']['count'] += 1

        try:
            p_sql = process_sql.get_sql(schema, p_str)
        except:
            # If p_sql is not valid, then we will use an empty sql to evaluate with the correct sql
            p_sql = {
            "except": None,
            "from": {
                "conds": [],
                "table_units": []
            },
            "groupBy": [],
            "having": [],
            "intersect": None,
            "limit": None,
            "orderBy": [],
            "select": [
                False,
                []
            ],
            "union": None,
            "where": []
            }
            eval_err_num += 1
            print("eval_err_num:{}".format(eval_err_num))

        # rebuild sql for value evaluation
        kmap = kmaps[db_name]
        g_valid_col_units = evaluation.build_valid_col_units(g_sql['from']['table_units'], schema)
        g_sql = evaluation.rebuild_sql_val(g_sql)
        g_sql = evaluation.rebuild_sql_col(g_valid_col_units, g_sql, kmap)
        p_valid_col_units = evaluation.build_valid_col_units(p_sql['from']['table_units'], schema)
        p_sql = evaluation.rebuild_sql_val(p_sql)
        p_sql = evaluation.rebuild_sql_col(p_valid_col_units, p_sql, kmap)

#         if etype in ["all", "exec"]:
#             exec_score = evaluation.eval_exec_match(db, p_str, g_str, p_sql, g_sql)
#             if exec_score:
#                 scores[hardness]['exec'] += 1

        if etype in ["all", "match"]:
            exact_score = evaluator.eval_exact_match(p_sql, g_sql)
            partial_scores = evaluator.partial_scores
#             if exact_score == 0:
#                 print("{} pred: {}".format(hardness,p_str))
#                 print("{} gold: {}".format(hardness,g_str))
#                 print("")
            scores[hardness]['exact'] += exact_score
            scores['all']['exact'] += exact_score
            for type_ in partial_types:
                if partial_scores[type_]['pred_total'] > 0:
                    scores[hardness]['partial'][type_]['acc'] += partial_scores[type_]['acc']
                    scores[hardness]['partial'][type_]['acc_count'] += 1
                if partial_scores[type_]['label_total'] > 0:
                    scores[hardness]['partial'][type_]['rec'] += partial_scores[type_]['rec']
                    scores[hardness]['partial'][type_]['rec_count'] += 1
                scores[hardness]['partial'][type_]['f1'] += partial_scores[type_]['f1']
                if partial_scores[type_]['pred_total'] > 0:
                    scores['all']['partial'][type_]['acc'] += partial_scores[type_]['acc']
                    scores['all']['partial'][type_]['acc_count'] += 1
                if partial_scores[type_]['label_total'] > 0:
                    scores['all']['partial'][type_]['rec'] += partial_scores[type_]['rec']
                    scores['all']['partial'][type_]['rec_count'] += 1
                scores['all']['partial'][type_]['f1'] += partial_scores[type_]['f1']

            # Custom
            partial_summary_score = sum([partial_scores[tp]['f1'] * max(partial_scores[tp]['label_total'], partial_scores[tp]['pred_total']) for tp in partial_types]) / sum([max(partial_scores[tp]['label_total'], partial_scores[tp]['pred_total']) for tp in partial_types])
            scores[hardness]['partial_summary'] += partial_summary_score
            scores['all']['partial_summary'] += partial_summary_score
                
            entries.append({
                'predictSQL': p_str,
                'goldSQL': g_str,
                'hardness': hardness,
                'exact': exact_score,
                'partial': partial_scores
            })

    for level in levels:
        if scores[level]['count'] == 0:
            continue
        if etype in ["all", "exec"]:
            scores[level]['exec'] /= scores[level]['count']

        if etype in ["all", "match"]:
            scores[level]['exact'] /= scores[level]['count']
            scores[level]['partial_summary'] /= scores[level]['count']
            for type_ in partial_types:
                if scores[level]['partial'][type_]['acc_count'] == 0:
                    scores[level]['partial'][type_]['acc'] = 0
                else:
                    scores[level]['partial'][type_]['acc'] = scores[level]['partial'][type_]['acc'] / \
                                                             scores[level]['partial'][type_]['acc_count'] * 1.0
                if scores[level]['partial'][type_]['rec_count'] == 0:
                    scores[level]['partial'][type_]['rec'] = 0
                else:
                    scores[level]['partial'][type_]['rec'] = scores[level]['partial'][type_]['rec'] / \
                                                             scores[level]['partial'][type_]['rec_count'] * 1.0
                if scores[level]['partial'][type_]['acc'] == 0 and scores[level]['partial'][type_]['rec'] == 0:
                    scores[level]['partial'][type_]['f1'] = 1
                else:
                    scores[level]['partial'][type_]['f1'] = \
                        2.0 * scores[level]['partial'][type_]['acc'] * scores[level]['partial'][type_]['rec'] / (
                        scores[level]['partial'][type_]['rec'] + scores[level]['partial'][type_]['acc'])

    evaluation.print_scores(scores, etype)
    print('================   PARTIAL MATCHING SUMMARY SCORE    ================')
    this_scores = [scores[level]['partial_summary'] for level in levels]
    print("{:20} {:<20.3f} {:<20.3f} {:<20.3f} {:<20.3f} {:<20.3f}".format("partial_summary", *this_scores))
    print()
    print('\t'.join([str(scores[level]['exact']) for level in levels]))
    print('\t'.join([str(scores[level]['partial_summary']) for level in levels]))
    # return scores


#### Basic compares

In [ ]:
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'

In [ ]:
# human test 
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

In [ ]:
with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

In [ ]:
# Just using the 1st ASR candidate, no correction 

ref_list = []
hyp_list = []

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
        
    c = d[0]
        
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    _db_id = o['db_id']
    
    _pred_sql = Question(c['question'], _db_id, model_dict=rat_sql_asr_model_dict)[0]['inferred_code']
    
    _gold_sql = c['query']
    _exact, _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
    
    c['pred_sql'] = _pred_sql
    c['score'] = _score
    c['exact'] = _exact

    _question_toks = [_t.lower() for _t in c['question_toks']]
    _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]
    ref_list.append([_gold_question_toks])
    hyp_list.append(_question_toks)


In [ ]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
_avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

## BLEU 
_bleu = corpus_bleu(list_of_references=ref_list,
                    hypotheses=hyp_list)

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))
print(f'BLEU = {_bleu:.4f}')


In [ ]:
EvaluateSQL_full(plist=[d[0]['pred_sql'] for d in test_dataset],
                 glist=[d[0]['query'] for d in test_dataset],
                 db_id_list=[d[0]['db_id'] for d in test_dataset],
                 kmaps=kmaps)

In [ ]:
# Dump the predictions 
# Non ASR
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'
# ASR
test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/First-cands.json'

with open(test_output_path, 'w') as f:
    json.dump(test_dataset, f, indent=4)

In [ ]:
# Using original text (no ASR)

with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

In [ ]:
# Using original text (no ASR)

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
        
    c = d[0]
        
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    _db_id = o['db_id']
    
#     _pred_sql = Question(c['gold_question'], _db_id, model_dict=rat_sql_asr_model_dict)[0]['inferred_code']
    _pred_sql = Question(c['gold_question'], _db_id)[0]['inferred_code']
    
    _gold_sql = c['query']
    _exact, _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
    
    c['pred_sql'] = _pred_sql
    c['score'] = _score
    c['exact'] = _exact


In [ ]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
_avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))


In [ ]:
EvaluateSQL_full(plist=[d[0]['pred_sql'] for d in test_dataset],
                 glist=[d[0]['query'] for d in test_dataset],
                 db_id_list=[d[0]['db_id'] for d in test_dataset],
                 kmaps=kmaps)

In [ ]:
random_subset = random.sample(test_dataset, k=100)

EvaluateSQL_full(plist=[d[0]['pred_sql'] for d in random_subset],
                 glist=[d[0]['query'] for d in random_subset],
                 db_id_list=[d[0]['db_id'] for d in random_subset],
                 kmaps=kmaps
                )

#### Tagger-ILM

In [ ]:
def Postprocess_rewrite_seq(tags, rewrite_seq, question_toks):
    _question_toks_placeholders = []

    for i, tok in enumerate(question_toks):
        if tags[i].endswith('KEEP'):
            _question_toks_placeholders.append(tok)
        elif (tags[i] == 'U-EDIT') or (tags[i] == 'B-EDIT'):
            _question_toks_placeholders.append('[EDIT]')
        elif (tags[i] == 'I-EDIT') or (tags[i] == 'L-EDIT') or tags[i].endswith('DEL'):
            pass
        else:
            print('Unknown tag: {}'.format(tags[i]))

    _edits = []
    _curr_edit = []
    for tok in rewrite_seq:
        if tok == '[ANS]':
            _edits.append(_curr_edit)
            _curr_edit = []
        elif tok == '@end@':  # Allennlp END_SYMBOL 
            break
        else:
            _curr_edit.append(tok)
    
    _question_toks_rewritten = []
    _edit_idx = 0
    for tok in _question_toks_placeholders:
        if tok == '[EDIT]':
            if _edit_idx >= len(_edits):
                print('--- Not enough edits ---')
                print('Tags:', tags)
                print('Edits:', _edits)
            else:
                _question_toks_rewritten.extend(_edits[_edit_idx])
            _edit_idx += 1
        else:
            _question_toks_rewritten.append(tok)

    return _question_toks_rewritten

In [ ]:
VERSION = '2.6.0.2t-2.6.0.2i'

rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'


In [ ]:
# human test 
VERSION = '2.6.0.2t-2.6.0.2i'

rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'


In [ ]:
with open(rewriter_ILM_pred_path, 'r') as f:
    rewriter_ILM_preds = [json.loads(l) for l in f.readlines()]
with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(rewriter_ILM_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

In [ ]:
for _p in rewriter_ILM_preds[::100]:
    print(_p['rewrite_seq_prediction'])

In [ ]:
# Quick evaluation: only using the 1st ASR candidate

pred_idx = 0

ref_list = []
hyp_list = []

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
        
    c = d[0]
        
    p = rewriter_ILM_preds[pred_idx]
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    # Debug 
    # assert c['rewriter_tags'] == p['rewriter_tags'][:len(c['rewriter_tags'])], f"{c['rewriter_tags']}\n{p['rewriter_tags']}\nShould raise"

    _db_id = o['db_id']

    # _tags = p['tags_prediction']  # For previous taggerILM joint model 
    # _tags = p['tags']  # Before adding align_tags (when 'tags' refers to 'rewriter_tags')
    
    _tags = p['rewriter_tags']
    _rewrite_seq = p['rewrite_seq_prediction']
    _question_toks = c['question_toks']

    _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
    _rewritten_question = ' '.join(_rewritten_question_toks)

    # _pred_sql = Question(_rewritten_question, _db_id, model_dict=rat_sql_asr_model_dict)[0]['inferred_code']
    _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']

    _gold_sql = c['query']
    _exact, _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

    # Save the taggerILM raw outputs, for later aggregation 
    c['pred_tags'] = p['rewriter_tags']
    c['pred_ILM'] = p['rewrite_seq_prediction']
    c['pred_ILM_cands'] = p['rewrite_seq_prediction_cands']
    
    # Save prediction results 
    c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
    c['pred_sql'] = p['pred_sql'] = _pred_sql
    p['gold_sql'] = _gold_sql
    c['score'] = p['score'] = _score
    c['exact'] = p['exact'] = _exact

    _rewritten_question_toks = [_t.lower() for _t in _rewritten_question_toks]
    _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

    ref_list.append([_gold_question_toks])
    hyp_list.append(_rewritten_question_toks)
    
    pred_idx += len(d)

    

In [ ]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
_avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

## BLEU 
_bleu = corpus_bleu(list_of_references=ref_list,
                    hypotheses=hyp_list)

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))
print(f'BLEU = {_bleu:.4f}')


In [ ]:
# Non ASR
test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'
# ASR
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/{VERSION}.json'

with open(test_output_path, 'w') as f:
    json.dump(test_dataset, f, indent=4)

#### Loading from predicted file (only 1st cand is predicted!)

In [ ]:
VERSION = '3.6.0.2'

test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

with open(test_output_path, 'r') as f:
    test_dataset = json.load(f)

len(test_dataset), test_dataset[0][0].keys()

In [ ]:
# Using EvaluateSQL_full 

# tables_json = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/tables.json'
# kmaps = evaluation.build_foreign_key_map_from_json(tables_json)

plist = [d[0]['pred_sql'] for d in test_dataset]
glist = [d[0]['query'] for d in test_dataset]
db_id_list = [d[0]['db_id'] for d in test_dataset]

EvaluateSQL_full(glist=glist,
                 plist=plist,
                 db_id_list=db_id_list,
                 kmaps=kmaps)


In [ ]:
# For reranker output file 
VERSION = '1.10.0.2'

test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

with open(test_output_path, 'r') as f:
    test_dataset = json.load(f)

len(test_dataset), test_dataset[0][0].keys()

In [ ]:
plist = [c['ratsql_pred_sql'] for d in test_dataset for c in d if c['is_reranker_selection'] == 1]
glist = [d[0]['query'] for d in test_dataset]
db_id_list = [d[0]['db_id'] for d in test_dataset]
assert len(plist) == len(glist) == len(db_id_list)

EvaluateSQL_full(glist=glist,
                 plist=plist,
                 db_id_list=db_id_list,
                 kmaps=kmaps)

In [ ]:
plist = [d[0]['ratsql_pred_sql'] for d in test_dataset]
glist = [d[0]['query'] for d in test_dataset]
db_id_list = [d[0]['db_id'] for d in test_dataset]
assert len(plist) == len(glist) == len(db_id_list)

EvaluateSQL_full(glist=glist,
                 plist=plist,
                 db_id_list=db_id_list,
                 kmaps=kmaps)

In [ ]:
# Using all ASR candidates (no longer in use)

pred_idx = 0

for d in tqdm(test_dataset):
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _db_id = o['db_id']

        # _tags = p['tags_prediction']
        _tags = p['tags']
        _rewrite_seq = p['rewrite_seq_prediction']
        _question_toks = c['question_toks']
        
        _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
        _rewritten_question = ' '.join(_rewritten_question_toks)
        
        _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
        
        _gold_sql = c['query']
        _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
        
        c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score

        pred_idx += 1



In [ ]:
# Using all the candidates to rewrite 
print(sum([p['score'] for p in rewriter_ILM_preds]) / len(rewriter_ILM_preds))
print(sum([c['score'] for d in test_dataset for c in d]) / sum([len(d) for d in test_dataset]))

# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))

In [ ]:
# Evaluation process with oracle tags (no longer in use for version>=2.3.0)

pred_idx = 0

for d in tqdm(test_dataset):
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _db_id = o['db_id']

        _tags = p['gold_tags']
        _rewrite_seq = p['oracle_tags_rewrite_seq_prediction']
        _question_toks = c['question_toks']
        
        _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
        _rewritten_question = ' '.join(_rewritten_question_toks)
        
        _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
        
        _gold_sql = c['query']
        _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
        
        c['oracle_tags_rewritten_question'] = p['oracle_tags_rewritten_question'] = _rewritten_question
        c['oracle_tags_pred_sql'] = p['oracle_tags_pred_sql'] = _pred_sql
        c['oracle_tags_score'] = p['oracle_tags_score'] = _score

        pred_idx += 1



In [ ]:
# Using all the candidates to rewrite 
print(sum([p['oracle_tags_score'] for p in rewriter_ILM_preds]) / len(rewriter_ILM_preds))
print(sum([c['oracle_tags_score'] for d in test_dataset for c in d]) / sum([len(d) for d in test_dataset]))

# Only using the 1st candidate to rewrite 
_oracle_avg_1st = sum([d[0]['oracle_tags_score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_oracle_std_1st = np.std([d[0]['oracle_tags_score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_oracle_avg_1st, _oracle_std_1st))

In [ ]:
# Merge results in a single dataset obj 

test_pred_dataset = []

pred_idx = 0

for d in tqdm(test_dataset):
    _pred_d = []
    
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _pred_c = dict()
        
        _pred_c['ASR_question'] = p['question']
        _pred_c['ASR_question_pred_sql'] = c['ratsql_pred_sql']
        
        _pred_c['gold_question'] = c['gold_question']
        # _pred_c['gold_question_pred_sql'] = orig_dev_preds[c['original_id']]
        
        _pred_c['tag_prediction'] = list(zip(p['question'].split(' '), p['tags_prediction']))
        _pred_c['rewrite_seq'] = []
        for t in p['rewrite_seq_prediction']:
            _pred_c['rewrite_seq'].append(t)
            if t == '@end@': break
        _pred_c['rewritten_question'] = p['rewritten_question']
        _pred_c['pred_sql'] = p['pred_sql']
        _pred_c['score'] = p['score']
        
        _pred_c['gold_tags'] = list(zip(p['question'].split(' '), p['gold_tags']))
        _pred_c['oracle_tags_rewrite_seq'] = []
        for t in p['oracle_tags_rewrite_seq_prediction']:
            _pred_c['oracle_tags_rewrite_seq'].append(t)
            if t == '@end@': break
        _pred_c['oracle_tags_rewritten_question'] = p['oracle_tags_rewritten_question']
        _pred_c['oracle_tags_pred_sql'] = p['oracle_tags_pred_sql']
        _pred_c['oracle_tags_score'] = p['oracle_tags_score']
        
        _pred_c['gold_sql'] = c['query']
        
        _pred_d.append(_pred_c)

        pred_idx += 1
    
    test_pred_dataset.append(_pred_d)

len(test_pred_dataset), sum([len(d) for d in test_pred_dataset])

In [ ]:
with open('./output/test-prediction-{}.json'.format(VERSION), 'w') as f:
    json.dump(test_pred_dataset, f, indent=4)

In [ ]:
# Load dataset file with predictions 

with open('./output/pred-{}.json'.format(VERSION), 'r') as f:
    test_pred_dataset = json.load(f)
len(test_pred_dataset), sum([len(d) for d in test_pred_dataset])

In [ ]:
test_pred_dataset[0][0].keys()

In [ ]:
# Analysis 
orig_dev_preds_path = './output/dev_output.txt'

with open(orig_dev_preds_path, 'r') as f:
    orig_dev_preds = [l.strip() for l in f.readlines()]

len(orig_dev_preds)

In [ ]:
# for d in test_dataset[5::40]:
#     print('DB:', d[0]['db_id'])
#     print('ASR question:\t\t', d[0]['question'])
#     print('Rewritten question:\t', d[0]['rewritten_question'])
#     print('Gold question:\t\t', d[0]['gold_question'])
#     print('ASR-q Pred SQL:\t\t', d[0]['ratsql_pred_sql'])
#     print('Rewritten-q Pred SQL:\t', d[0]['pred_sql'])
#     print('Gold-q Pred SQL:\t', orig_dev_preds[d[0]['original_id']])
#     print('Gold SQL:\t\t', d[0]['query'])
#     print('Score:', d[0]['score'])
#     print('-'*50)

In [ ]:
test_pred_samples = [c for d in test_pred_dataset for c in d]
for i, c in list(enumerate(test_pred_samples))[8::88]:
    print('-'*30, 'ID = {}'.format(i), '-'*30)
    print('ASR question:\t\t', c['ASR_question'])
    print('Rewritten question:\t', c['rewritten_question'])
    print('Gold question:\t\t', c['gold_question'])
    print('Rewritten-q Pred SQL:\t', c['pred_sql'])
    print('Gold-q Pred SQL:\t', c['gold_question_pred_sql'])
    print('Gold SQL:\t\t', c['gold_sql'])
    print('Score:', c['score'])
    

In [ ]:
inspect_ids = [8, 96, 272, 448, 1416, 1592, 1680, 1856, 2120, 2296, 2384, 2560, 2824]

#### Seq2seq

In [ ]:
## Batch evaluating 

VERSION_LIST = ['3.1.1_ep0', '3.1.1.1_ep0', '3.1.1.2_ep0']
HUMAN_TEST = True

for VERSION in VERSION_LIST:
    print(f'Evaluating{" (human test)" if HUMAN_TEST else ""}: VERSION = {VERSION}')
    
    if not HUMAN_TEST:
        rewriter_s2s_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{VERSION}.json'
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    else:
        # human test 
        rewriter_s2s_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

    
    with open(rewriter_s2s_pred_path, 'r') as f:
        rewriter_preds = [json.loads(l) for l in f.readlines()]
    with open(test_dataset_path, 'r') as f:
        test_dataset = json.load(f)
    with open(orig_dev_path, 'r') as f:
        orig_dev_dataset = json.load(f)
        
    # Quick evaluation: only using the 1st ASR candidate

    ref_list = []
    hyp_list = []
    
    pred_idx = 0

    for d in tqdm(test_dataset):
        if len(d) == 0:
            continue

        c = d[0]

        p = rewriter_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

        _db_id = o['db_id']

        # _tags = p['tags']
        # _rewrite_seq = p['rewrite_seq_prediction']
        # _question_toks = c['question_toks']

        # _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
        # _rewritten_question = ' '.join(_rewritten_question_toks)

        _rewritten_question = ' '.join(p['s2s_prediction'])

        if _rewritten_question == '':
            print(f'_rewritten_question is empty')
            _pred_sql = ''
            _gold_sql = c['query']
            _score = 0
        else:
            _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
            _gold_sql = c['query']
            _exact, _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

        c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score
        c['exact'] = p['exact'] = _exact
        
        # For BLEU 
        _rewritten_question_toks = [_t.lower() for _t in p['s2s_prediction']]
        _question_toks = [_t.lower() for _t in c['question_toks']]
        _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

        ref_list.append([_gold_question_toks])
        hyp_list.append(_rewritten_question_toks)

        pred_idx += len(d)

    # Only using the 1st candidate to rewrite 
    _avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
    _avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)

    ## Std-dev (1st cand only)
    _std_1st = np.std([d[0]['score'] for d in test_dataset])
    
    ## BLEU 
    _bleu = corpus_bleu(list_of_references=ref_list,
                        hypotheses=hyp_list)
    
    print(f'VERSION {VERSION}:')
    print(f'avg = {_avg_1st:.4f} (std = {_std_1st:.4f})')
    print(f'avg_exact = {_avg_exact_1st:.4f}')
    print(f'BLEU = {_bleu:.4f}')
    print()
    
    if not HUMAN_TEST:
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

        with open(test_output_path, 'w') as f:
            json.dump(test_dataset, f, indent=4)


In [ ]:
VERSION = '3.3.0.0'

rewriter_s2s_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{VERSION}.json'
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'


In [ ]:
with open(rewriter_s2s_pred_path, 'r') as f:
    rewriter_preds = [json.loads(l) for l in f.readlines()]
with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(rewriter_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

In [ ]:
rewriter_preds[0].keys()

In [ ]:
for p in rewriter_preds[3::300]:
    print(p['question'])
    print(' '.join(p['s2s_prediction']))
    print(' '.join(p['gold_rewrite_seq_s2s'][1:-1]))
    print('-'*50)

In [ ]:
# Quick evaluation: only using the 1st ASR candidate

pred_idx = 0

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
        
    c = d[0]
    
    p = rewriter_preds[pred_idx]
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

    _db_id = o['db_id']

    # _tags = p['tags']
    # _rewrite_seq = p['rewrite_seq_prediction']
    # _question_toks = c['question_toks']

    # _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
    # _rewritten_question = ' '.join(_rewritten_question_toks)
    
    _rewritten_question = ' '.join(p['s2s_prediction'])
    
    if _rewritten_question == '':
        print(f'_rewritten_question is empty')
        _pred_sql = ''
        _gold_sql = c['query']
        _score = 0
    else:
        _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
        _gold_sql = c['query']
        _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

    c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
    c['pred_sql'] = p['pred_sql'] = _pred_sql
    p['gold_sql'] = _gold_sql
    c['score'] = p['score'] = _score

    pred_idx += len(d)

    

In [ ]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))

In [ ]:
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

# with open(test_output_path, 'w') as f:
#     json.dump(test_dataset, f, indent=4)

In [ ]:
# Actual (full) evaluation process 

pred_idx = 0

for d in tqdm(test_dataset):
    for c in d:
        p = rewriter_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        pred_idx += 1
        if 'score' in c:
            continue  # already inferred  
        
        _db_id = o['db_id']

        _rewritten_question = ' '.join(p['s2s_prediction'])
        _pred_result = Question(_rewritten_question, _db_id)
        
        _gold_sql = c['query']
        
        if len(_pred_result) == 0:
            print(_db_id, _rewritten_question, '-- no predictiction')
            _pred_sql = ''
            _score = 0
        else:
            _pred_sql = _pred_result[0]['inferred_code']
            _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
        
        c['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score

        


In [ ]:
# Using all the candidates to rewrite 
print(sum([p['score'] for p in rewriter_preds]) / len(rewriter_preds))
print(sum([c['score'] for d in test_dataset for c in d]) / sum([len(d) for d in test_dataset]))

# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))

In [ ]:
# Merge results in a single dataset obj 

test_pred_dataset = []

pred_idx = 0

for d in tqdm(test_dataset):
    _pred_d = []
    
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _pred_c = dict()
        
        _pred_c['ASR_question'] = p['question']
        _pred_c['ASR_question_pred_sql'] = c['ratsql_pred_sql']
        
        _pred_c['gold_question'] = c['gold_question']
        # _pred_c['gold_question_pred_sql'] = orig_dev_preds[c['original_id']]
        
        _pred_c['rewritten_question'] = p['s2s_prediction']
        _pred_c['pred_sql'] = p['pred_sql']
        _pred_c['score'] = p['score']
        
        _pred_c['gold_sql'] = c['query']
        
        _pred_d.append(_pred_c)

        pred_idx += 1
    
    test_pred_dataset.append(_pred_d)

len(test_pred_dataset), sum([len(d) for d in test_pred_dataset])

In [ ]:
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

# with open(test_output_path, 'w') as f:
#     json.dump(test_dataset, f, indent=4)

In [ ]:
test_samples = [c for d in test_dataset for c in d]
for i, c in list(enumerate(test_samples))[8::88]:
    print('-'*30, 'ID = {}'.format(i), '-'*30)
    print('ASR question:\t\t', c['question'])
    print('Rewritten question:\t', c['rewritten_question'])
    print('Gold question:\t\t', c['gold_question'])
    print('Rewritten-q Pred SQL:\t', c['pred_sql'])
#     print('Gold-q Pred SQL:\t', c['gold_question_pred_sql'])
    print('Gold SQL:\t\t', c['query'])
    print('Score:', c['score'])
    

### Temp

In [ ]:
rewriter_ILM_preds[0].keys()

In [ ]:
test_dataset[0][0].keys()

In [ ]:
orig_dev_dataset[0].keys()

In [ ]:
# Rewrite_seq postprocessing, to get the rewritten question 

_idx = 154

_tags = rewriter_ILM_preds[_idx]['tags_prediction']
_rewrite_seq = rewriter_ILM_preds[_idx]['rewrite_seq_prediction']
_question_toks = rewriter_ILM_preds[_idx]['question'].split(' ')
_tags, _rewrite_seq, _question_toks

postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)

In [ ]:
db = 'concert_singer'
g_str = 'SELECT count(*) FROM singer'
p_str = "SELECT Count(DISTINCT singer.Name) FROM singer WHERE singer.Name = 'terminal'"

db, p_str, g_str, EvaluateSQL(p_str, g_str, db)

In [ ]:
%load_ext memory_profiler

In [ ]:
%memit